# Background Information

In this notebook I will set up a basic flow for uploading and dowloading files. Formating datasets and ultimately running cytetype.

In [3]:
# Importing Require Libraries
import cytetype
import scanpy as sc
import session_info
import pandas as pd

In [4]:
# Session Info
session_info.show()

/Users/alexantill/Göran_Karlsson_Lab/code_benchLLM/.venv/lib/python3.12/site-packages/session_info/main.py:213: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  mod_version = _find_version(mod.__version__)


In [ ]:
# Importing AnnData file
# File Path
file_path_Granja = os.path.expanduser("~/Göran_Karlsson_Lab/benchLLM/Simone_et_al/7d821d98-5b42-4480-8173-641c1b37b237.h5ad")
# AnnData Read In
adata = sc.read_h5ad(file_path_Granja)

In [ ]:
# Copy and Preprocess the Original AnnData
adata_norm = adata.copy()

# Normalize (10,000 Counts per Cell)
sc.pp.normalize_total(adata_norm, target_sum=1e4)

# Log1p transform
sc.pp.log1p(adata_norm)

# Refine to 2000 Highly Varibale Genes Only
sc.pp.highly_variable_genes(adata_norm, n_top_genes=2000, flavor='seurat')
adata = adata_norm[:, adata_norm.var.highly_variable]

In [ ]:
# Find Most Up/Down Regulated Genes 
sc.tl.rank_genes_groups(adata, groupby='BioClassification', method='t-test')  # try 'wilcoxon' if accurate

result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names

top_genes_per_cluster = {group: result['names'][group][:10] for group in groups}
print(top_genes_per_cluster)